## import, read static files

In [281]:
import pandas as pd, numpy as np
import statsapi
from statsapi import player_stat_data
import requests
from datetime import datetime, timedelta
import numpy as np
import math
import sqlalchemy
import mysql.connector
import meteostat
import pprint
import pickle
import threading

In [129]:
Teams_and_IDs = pd.read_csv("Teams_and_IDs.csv")
Player_Positions = pd.read_csv("MLB_Player_Positions.csv")
Park_Data = pd.read_csv("mlb_parks.csv")
park_venues = set(Park_Data["Venue"])

In [3]:
All_Teams_Data = []

for team_id in Teams_and_IDs['Team_ID']:
    roster_url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/roster?rosterType=allTime"
    response = requests.get(roster_url)
    roster_data = response.json()['roster']
    for player in roster_data:
        Player_Name = player['person']['fullName']
        Player_ID = player['person']['id']
        Player_Dictionary = {"Team_ID":team_id,"Player Name":Player_Name, "Player ID":Player_ID}
        All_Teams_Data.append(Player_Dictionary)

All_Teams_DataFrame = pd.DataFrame(All_Teams_Data)

In [4]:
# almost the same could be achieve this way.
All_Teams_DataFrame_ = Player_Positions.drop_duplicates(['player_name', 'player_id', 'player_team_id'])[['player_team_id', 'player_name', 'player_id']]

In [5]:
def Team_To_ID(team_name):
    team_id = Teams_and_IDs["Team_ID"][Teams_and_IDs['Team_Name'] == team_name].iloc[0]
    return team_id

def ID_To_Team(team_id):
    team_name = Teams_and_IDs["Team_Name"][Teams_and_IDs['Team_ID'] == team_id].iloc[0]
    return team_name

def Player_to_ID(player_name):
    player_id = Player_Positions["player_id"][Player_Positions["player_name"] == player_name]
    if len(player_id) >= 1:
        return player_id.iloc[0]
    else:
        return ""

def celsius_to_fahrenheit(celsius):
    return ( celsius * (9/5) ) + 32    

## peep at the api

In [243]:
tomorrow = (datetime.today() + timedelta(days = 1)).strftime("%Y-%m-%d")
yesterday = (datetime.today() - timedelta(days = 1)).strftime("%Y-%m-%d")
day_before_yesterday = (datetime.today() - timedelta(days = 2)).strftime("%Y-%m-%d")

# Schedule = statsapi.schedule(start_date = day_before_yesterday, end_date = yesterday)
Schedule = statsapi.schedule(start_date = "2011-04-01", end_date = "2020-03-01")
Schedule_DataFrame = pd.json_normalize(Schedule)

game_id_list = list(Schedule_DataFrame["game_id"].drop_duplicates())
start = datetime.now()

Excluding seriesStatus hydration because the MLB API throws an error for 2014-03-11 which is included in the requested date range.


In [245]:
Schedule_DataFrame[['game_datetime', 'game_date', 'home_probable_pitcher', 'away_probable_pitcher',
       'away_score', 'home_score',
       'current_inning', 'inning_state', 'status']].tail(2)

,game_datetime,game_date,home_probable_pitcher,away_probable_pitcher,away_score,home_score,current_inning,inning_state,status
2987,2012-04-01T20:05:00Z,2012-04-01,Yovani Gallardo,Tim Lincecum,4,7,9,Top,Final
2988,2012-04-01T20:10:00Z,2012-04-01,Jeremy Guthrie,Ubaldo Jimenez,12,10,9,Bottom,Final


In [246]:
Schedule_DataFrame[['game_datetime', 'game_date', 'home_probable_pitcher', 'away_probable_pitcher',
       'away_score', 'home_score',
       'current_inning', 'inning_state', 'status']].head(2)

,game_datetime,game_date,home_probable_pitcher,away_probable_pitcher,away_score,home_score,current_inning,inning_state,status
0,2011-04-01T17:05:00Z,2011-04-01,Roy Halladay,Brett Myers,4,5,9,Bottom,Final
1,2011-04-01T18:20:00Z,2011-04-01,Ryan Dempster,Kevin Correia,6,3,9,Bottom,Final


# collect and cache history data

## schedules

In [56]:
_schedules = pickle.load(open('schedules.pkl', 'rb'))

In [268]:
def get_end_date(year):
    end_date = f"{year+1}-03-01"
    if year == 2022:
        end_date = f"{year}-12-01"
    elif year == 2023:
        end_date = (datetime.today() - timedelta(days = 1)).strftime("%Y-%m-%d")
    return end_date

In [260]:
def fetch_schdules_year(year):
    print(f'fetch_schdules_year {year}')
    global _schedules
    data = None

    schedule = statsapi.schedule(start_date = f"{year}-04-01", end_date = get_end_date(year))
    for schedul in schedule:
        _schedules[schedul['game_id']] = schedul

    print(f'done fetch_schdules_year {year}')

In [264]:
fetch_schdules_year(2023)

fetch_schdules_year 2023
done fetch_schdules_year 2023


In [263]:
len(_schedules)

28937

In [265]:
len(_schedules)

29650

In [261]:
'''
fetch_schdules_year(2022)
fetch_schdules_year(2021)
fetch_schdules_year(2020)
fetch_schdules_year(2019)
fetch_schdules_year(2018)
fetch_schdules_year(2017)
fetch_schdules_year(2016)
fetch_schdules_year(2015)
fetch_schdules_year(2014)
fetch_schdules_year(2013)
fetch_schdules_year(2012)
fetch_schdules_year(2011)
#'''

'\nfetch_schdules_year(2022)\nfetch_schdules_year(2021)\nfetch_schdules_year(2020)\nfetch_schdules_year(2019)\nfetch_schdules_year(2018)\nfetch_schdules_year(2017)\nfetch_schdules_year(2016)\nfetch_schdules_year(2015)\nfetch_schdules_year(2014)\nfetch_schdules_year(2013)\nfetch_schdules_year(2012)\nfetch_schdules_year(2011)\n#'

In [267]:
pickle.dump(_schedules, open('schedules.pkl', 'wb'))

## game ids & boxscores

In [ ]:
_boxscores = pickle.load(open('boxscores.pkl', 'rb'))
_boxscores_lock = threading.Lock()

In [ ]:
def get_boxscore_data(game_id, force_fetch = False):
    global _boxscores
    _boxscores_lock.acquire()
    data = None
    if game_id not in _boxscores or force_fetch:
        try:
            #print(f'calling statsapi.boxscore_data for game_id: {game_id}')
            data = statsapi.boxscore_data(game_id)
        except Exception:
            print(f'Exception: {Exception} while fetching boxscore')
        if data is not None:
            _boxscores[game_id] = data
    elif game_id in _boxscores:
        data = _boxscores[game_id]
    
    _boxscores_lock.release()
    return data

def get_boxscore_data_batch(game_ids, force_fetch = False):
    print(f'{game_ids[:2]} total {len(game_ids)}')
    for i, game_id in enumerate(game_ids):
        if i % 100 == 0:
            print(f'{i} of {game_ids[:2]} total {len(game_ids)}')
        get_boxscore_data(game_id, force_fetch = force_fetch)
    return [get_boxscore_data(game_id, force_fetch = force_fetch) for game_id in game_ids]

def dump_boxscore_cache():
    global _boxscores
    pickle.dump(_boxscores, open('boxscores.pkl', 'wb'))

In [6]:
_game_id_lists = pickle.load(open('game_id_lists.pkl', 'rb'))

In [269]:
def get_game_id_list_year(year, force_fetch = False):
    print(f'getting game_id list {year}')
    global _game_id_lists
    data = None
    if year not in _game_id_lists or force_fetch:
        schedule = statsapi.schedule(start_date = f"{year}-04-01", end_date = get_end_date(year))
        df_schedule = pd.json_normalize(schedule)
        data = list(df_schedule["game_id"].drop_duplicates())
        if data is not None:
            _game_id_lists[year] = data
    elif year in _game_id_lists:
        data = _game_id_lists[year]
        
    print(f'done getting game_id list {year}')
    return data

def ingest_boxscore_year(year):
    print(f'year: {year}')
    game_id_list = get_game_id_list_year(year)

    game_id_list_splits = np.array_split(game_id_list, 8)
    ths = []
    for i, game_id_list_split in enumerate(game_id_list_splits):
        print(f'th {i}')
        th = threading.Thread(target=get_boxscore_data_batch, args=(game_id_list_split, ))
        th.start()
        ths.append(th)

    for th in ths:
        th.join()
    
    '''
    for i, game_id in enumerate(game_id_list):
        if i % 500 == 0:
            print(f'processing {i} out of {len(game_id_list)} game_id {game_id}')
        game_boxscore = get_boxscore_data(game_id)
    '''
    print(f'done ingest_boxscore_year {year}')    

In [ ]:
'''
get_game_id_list_year(2022)
get_game_id_list_year(2021)
get_game_id_list_year(2020)
get_game_id_list_year(2019)
get_game_id_list_year(2018)
get_game_id_list_year(2017)
get_game_id_list_year(2016)
get_game_id_list_year(2015)
get_game_id_list_year(2014)
get_game_id_list_year(2013)
get_game_id_list_year(2012)
get_game_id_list_year(2011)
#'''

In [274]:
pickle.dump(_game_id_lists, open('game_id_lists.pkl', 'wb'))

In [ ]:
'''
ingest_boxscore_year(2022)
ingest_boxscore_year(2021)
ingest_boxscore_year(2020)
ingest_boxscore_year(2019)
ingest_boxscore_year(2018)
ingest_boxscore_year(2017)
ingest_boxscore_year(2016)
ingest_boxscore_year(2015)
ingest_boxscore_year(2014)
ingest_boxscore_year(2013)
ingest_boxscore_year(2012)
ingest_boxscore_year(2011)
#'''

In [ ]:
dump_boxscore_cache()

In [275]:
len(_boxscores)

33511

## player_stats

In [85]:
_player_stats = pickle.load(open('player_stats.pkl', 'rb'))

In [87]:
def get_player_stat_data(player_id, force_fetch = False):
    global _player_stats
    data = None
    if player_id not in _player_stats or force_fetch:
        try:
            print(f'calling statsapi.player_stat_data for player_id: {player_id}')
            data = statsapi.player_stat_data(personId = player_id, group="hitting", type="yearByYear", sportId=1)
        except Exception:
            print(f'Exception: {Exception} while fetching player_stat_data')
        if data is not None:
            _player_stats[player_id] = data
    elif player_id in _player_stats:
        data = _player_stats[player_id]
    
    return data

def dump_player_stat_data_cache():
    global _player_stats
    pickle.dump(_player_stats, open('player_stats.pkl', 'wb'))

In [217]:
def get_matchup(game_id, side):
    game_boxscore = get_boxscore_data(game_id)
    if game_boxscore is None:
        print(f'Failed to get box score for game_id: {game_id}')
        return None

    game = _schedules[game_id]
    if game is None:
        print(f'Failed to get schedule detail for game_id: {game_id}')
        return None
    
    opposite_side = "away" if side == "home" else "home"
    all_side_players = game_boxscore[side]["players"]
    
    side_players_list = []
    for home_player in all_side_players:
        home_player = all_side_players[home_player]
        home_player_name, home_player_id = home_player["person"]["fullName"], home_player["person"]["id"]
        side_players_list.append({"name":home_player_name, "id":home_player_id})

    side_players_dataframe = pd.DataFrame(side_players_list)
    side_batting_lineup_ids = game_boxscore[side]["battingOrder"]
    side_batters = side_players_dataframe[side_players_dataframe["id"].isin(side_batting_lineup_ids)]
    valid_historical_season = int(game["game_date"][0:4]) - 1
    side_batter_stats_list = []

    for side_batter in side_batters["id"]:
        side_batter_stats_data = get_player_stat_data(side_batter, force_fetch = False)
        if side_batter_stats_data is not None:
            side_batter_stats = side_batter_stats_data["stats"]
        else:
            print(f'Error while getting {side} batter {side_batter} stat')
            return None

        if len(side_batter_stats) == 0:
            #print(f'{side} batter {side_batter} stat is empty')
            return None
        
        for historical_batter_stat in side_batter_stats:
            if historical_batter_stat["season"] == str(valid_historical_season):
                season_batter_stats = historical_batter_stat["stats"]
                season_batter_stats["name"] = side_batters["name"][side_batters["id"] == side_batter].iloc[0]
                season_batter_stats["id"] = side_batters["id"][side_batters["id"] == side_batter].iloc[0]
                
                side_batter_game_day_stats = all_side_players[f"ID{side_batter}"]["stats"]["batting"]
                
                if side_batter_game_day_stats["hits"] < 1:
                    hit_recorded = 0
                elif side_batter_game_day_stats["hits"] >= 1:
                    hit_recorded = 1
                    
                season_batter_stats["hit_recorded"] = hit_recorded
                side_batter_stats_list.append(season_batter_stats)
            
    side_team_batting_stats = pd.DataFrame(side_batter_stats_list).drop_duplicates(subset = "name", keep ="last")
    
    if len(side_team_batting_stats) < 1:
        print(f'{side} side_team_batting_stats is empty for game')
        return None
    
    opposing_pitcher_name, opposing_pitcher_id = game[f"{opposite_side}_probable_pitcher"], Player_to_ID(game[f"{opposite_side}_probable_pitcher"])
    
    side_batting_matchup = None
    if type(opposing_pitcher_id) != type(np.int64()):
        print(f'opposing_pitcher_id {opposing_pitcher_id} is not of int64 type')
        pass
    else:
        opposing_pitcher_stats_data = get_player_stat_data(opposing_pitcher_id, force_fetch = False)
        if opposing_pitcher_stats_data is not None:            
            opposing_pitcher_stats = pd.json_normalize(opposing_pitcher_stats_data["stats"], max_level = 0)
        else:
            print(f'Error while getting {side} opssosing pitcher {opposing_pitcher_id} stat')
            return None
        
        # If there is just no data from the API for this player
        if len(opposing_pitcher_stats) == 0:
            #print(f'{side} opposing pitcher {opposing_pitcher_id} stat is empty')
            pass
        else:
            valid_opposing_pitcher_season_stats = opposing_pitcher_stats[opposing_pitcher_stats["season"] == str(valid_historical_season)]["stats"].drop_duplicates(keep = "last")
        
            # If there is no historical data for last season
            if len(valid_opposing_pitcher_season_stats) == 0:
                print(f'{side} valid opposing pitcher {opposing_pitcher_id} season stat is empty')
                pass
            else:
                opposing_pitcher_season_stats = valid_opposing_pitcher_season_stats.iloc[0]
                opposing_pitcher_season_stats["name"] = opposing_pitcher_name
                opposing_pitcher_season_stats["id"] = opposing_pitcher_id
                
                opposing_pitcher_stats_dataframe = pd.DataFrame([opposing_pitcher_season_stats])
                opposing_pitcher_stats = pd.concat([opposing_pitcher_stats_dataframe]*len(side_team_batting_stats))
                side_batting_matchup = pd.concat([opposing_pitcher_stats.reset_index(drop = True).add_prefix("pitching_"), side_team_batting_stats.reset_index(drop = True).add_prefix("batting_")], axis = 1)
            
                # =============================================================================
                # End of calculating the side for the batters
                # =============================================================================
    return side_batting_matchup

In [88]:
def ingest_matchup_batch(game_ids, force_fetch = False):
    print(f'{game_ids[:2]} total {len(game_ids)}')

    for i, game_id in enumerate(game_ids):
        if i % 100 == 0:
            print(f'{i} of {game_ids[:2]} total {len(game_ids)}')
        home_batting_matchup = get_matchup(game_id, "home")
        away_batting_matchup = get_matchup(game_id, "away")
        
def ingest_matchup_year(year):
    print(f'ingest_matchup_year year: {year}')
    game_id_list = get_game_id_list_year(year)

    game_id_list_splits = np.array_split(game_id_list, 8)
    ths = []
    for i, game_id_list_split in enumerate(game_id_list_splits):
        print(f'th {i}')
        th = threading.Thread(target=ingest_matchup_batch, args=(game_id_list_split, ))
        th.start()
        ths.append(th)

    for th in ths:
        th.join()

    print(f'done ingest_matchup_year year: {year}')    

In [118]:
'''
ingest_matchup_year(2022)
ingest_matchup_year(2021)
ingest_matchup_year(2020)
ingest_matchup_year(2019)
ingest_matchup_year(2018)
ingest_matchup_year(2017)
ingest_matchup_year(2016)
ingest_matchup_year(2015)
ingest_matchup_year(2014)
ingest_matchup_year(2013)
ingest_matchup_year(2012)
ingest_matchup_year(2011)
#'''

'\ningest_matchup_year(2022)\ningest_matchup_year(2021)\ningest_matchup_year(2020)\ningest_matchup_year(2019)\ningest_matchup_year(2018)\ningest_matchup_year(2017)\ningest_matchup_year(2016)\ningest_matchup_year(2015)\ningest_matchup_year(2014)\ningest_matchup_year(2013)\ningest_matchup_year(2012)\ningest_matchup_year(2011)\n#'

In [ ]:
dump_player_stat_data_cache()

In [278]:
ingest_matchup_year(2023)

ingest_matchup_year year: 2023
getting game_id list 2023
done getting game_id list 2023
th 0
[718760 718759] total 90
0 of [718760 718759] total 90
th 1
[718670 718671] total 89
0 of [718670 718671] total 89
th 2
[718580 718583] total 89
0 of [718580 718583] total 89
th 3
[718491 718492] total 89
0 of [718491 718492] total 89
th 4
[718406 718409] total 89
0 of [718406 718409] total 89
th 5
[718317 718318] total 89
0 of [718317 718318] total 89
th 6
[718235 718227] total 89
0 of [718235 718227] total 89
th 7
[718143 718140] total 89
0 of [718143 718140] total 89
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
done ingest_matchup_year year: 2023


## venue_game_temperatures

In [147]:
_venue_game_temperatures = pickle.load(open('venue_game_temperatures.pkl', 'rb'))

In [193]:
def get_venue_game_temperatures(lat, lon, game_date, game_datetime):
    global _venue_game_temperatures

    key = str((lat, lon, game_datetime,))
    game_temperature = None
    if key not in _venue_game_temperatures:
        historical_weather = meteostat.Hourly(loc = meteostat.Point(lat = lat, lon = lon), start = (pd.to_datetime(game_date)), end = (pd.to_datetime(game_date) + timedelta(days = 1)), timezone = "America/Chicago").fetch().reset_index()
        pre_game_weather = historical_weather[historical_weather["time"] <= (pd.to_datetime(game_datetime)).tz_convert("America/Chicago")]
        last_hour_weather = pre_game_weather.tail(1).copy()
        game_temperature = celsius_to_fahrenheit(last_hour_weather["temp"].iloc[0])
    else:
        game_temperature = _venue_game_temperatures[key]

    return game_temperature

In [279]:
cnt_invalid_venue = 0
invalid_venues = set()
for i, game_id in enumerate(game_id_list_total):
    if i % 1000 == 0:
        print(f'processing {i} out of {len(game_id_list_total)} game_id {game_id} cnt_invalid_venue: {cnt_invalid_venue}, invalid_venues: {len(invalid_venues)}')
    game = _schedules[game_id]

    if game["venue_name"] not in park_venues:
        #print(f'game["venue_name"] {game["venue_name"]} not valid')
        cnt_invalid_venue += 1
        invalid_venues.add(game["venue_name"])
        continue

    game_temperature = get_venue_game_temperatures(park_lat, park_lon, game["game_date"], game["game_datetime"])
    key = str((park_lat, park_lon, game["game_datetime"],))

    _venue_game_temperatures[key] = game_temperature

print(f'cnt_invalid_venue {cnt_invalid_venue}, invalid_venues: {len(invalid_venues)}')

processing 0 out of 28937 game_id 286884 cnt_invalid_venue: 0, invalid_venues: 0


processing 1000 out of 28937 game_id 287899 cnt_invalid_venue: 307, invalid_venues: 9
processing 2000 out of 28937 game_id 288887 cnt_invalid_venue: 596, invalid_venues: 9
processing 3000 out of 28937 game_id 318217 cnt_invalid_venue: 904, invalid_venues: 38
processing 4000 out of 28937 game_id 319211 cnt_invalid_venue: 1180, invalid_venues: 38
processing 5000 out of 28937 game_id 362963 cnt_invalid_venue: 1459, invalid_venues: 39
processing 6000 out of 28937 game_id 347627 cnt_invalid_venue: 1834, invalid_venues: 45
processing 7000 out of 28937 game_id 348625 cnt_invalid_venue: 2143, invalid_venues: 45
processing 8000 out of 28937 game_id 380921 cnt_invalid_venue: 2465, invalid_venues: 47
processing 9000 out of 28937 game_id 381918 cnt_invalid_venue: 2770, invalid_venues: 47
processing 10000 out of 28937 game_id 382914 cnt_invalid_venue: 3064, invalid_venues: 47
processing 11000 out of 28937 game_id 414508 cnt_invalid_venue: 3393, invalid_venues: 55
processing 12000 out of 28937 game_

In [280]:
pickle.dump(_venue_game_temperatures, open('venue_game_temperatures.pkl', 'wb'))

# construct match up dataset

In [256]:
game_id_list_total = sum([get_game_id_list_year(y) for y in range(2011, 2023)], [])

getting game_id list 2011
done getting game_id list 2011
getting game_id list 2012
done getting game_id list 2012
getting game_id list 2013
done getting game_id list 2013
getting game_id list 2014
done getting game_id list 2014
getting game_id list 2015
done getting game_id list 2015
getting game_id list 2016
done getting game_id list 2016
getting game_id list 2017
done getting game_id list 2017
getting game_id list 2018
done getting game_id list 2018
getting game_id list 2019
done getting game_id list 2019
getting game_id list 2020
done getting game_id list 2020
getting game_id list 2021
done getting game_id list 2021
getting game_id list 2022
done getting game_id list 2022


In [257]:
len(game_id_list_total)

28937

In [258]:
game_id_list = get_game_id_list_year(2022)

getting game_id list 2022
done getting game_id list 2022


In [259]:
len(game_id_list)

2546

In [289]:
game_id_list_2023 = get_game_id_list_year(2023)

getting game_id list 2023
done getting game_id list 2023


In [220]:
game_matchups = []
cnt_null_matchups = 0
for i, game_id in enumerate(game_id_list_total):
    if i % 1000 == 0:
        print(f'processing {i} out of {len(game_id_list_total)} game_id {game_id}, cnt_null_matchups: {cnt_null_matchups}')
    game = _schedules[game_id]

    game_boxscore = get_boxscore_data(game_id)
    
    if game_boxscore is None:
        print(f'Failed to get box score for game_id: {game_id}')
        continue
    
    home_batting_matchup = get_matchup(game_id, "home")
    away_batting_matchup = get_matchup(game_id, "away")
    
    home_batting_matchup = home_batting_matchup if home_batting_matchup is not None else []
    away_batting_matchup = away_batting_matchup if away_batting_matchup is not None else []

    all_home_players = game_boxscore["home"]["players"]
    all_away_players = game_boxscore["away"]["players"]

    if (len(home_batting_matchup) == 0) and (len(away_batting_matchup) == 0):
        #print(f'None both home_batting_matchup and away_batting_matchup for game_id {game_id}')
        cnt_null_matchups += 1
        continue
    elif (len(home_batting_matchup) >= 1) and (len(away_batting_matchup) == 0):
        #print(f'None away_batting_matchup for game_id {game_id}')
        game_matchup = home_batting_matchup.reset_index(drop = True).copy()
    elif (len(home_batting_matchup) == 0) and (len(away_batting_matchup) >= 1):
        #print(f'None home_batting_matchup for game_id {game_id}')
        game_matchup = away_batting_matchup.reset_index(drop = True).copy()
    elif (len(home_batting_matchup) >= 0) and (len(away_batting_matchup) >= 1):
        game_matchup = pd.concat([home_batting_matchup, away_batting_matchup], axis = 0).reset_index(drop = True)

    game_matchup["game_id"] = game["game_id"]
    game_matchup["game_venue"] = game["venue_name"]
    game_matchup["game_date"] = game["game_date"]
    game_matchup["game_datetime"] = game["game_datetime"]

    if game["venue_name"] not in park_venues:
        # likely a training game
        continue

    park_lat = Park_Data["latitude"][Park_Data["Venue"] == game["venue_name"]].iloc[0]
    park_lon = Park_Data["longitude"][Park_Data["Venue"] == game["venue_name"]].iloc[0]

    game_temperature = get_venue_game_temperatures(park_lat, park_lon, game["game_date"], game["game_datetime"])
    game_matchup["temp"] = game_temperature
    game_matchups.append(game_matchup)

print(f'cnt_null_matchups: {cnt_null_matchups}')

processing 0 out of 28937 game_id 286884, cnt_null_matchups: 0
processing 1000 out of 28937 game_id 287899, cnt_null_matchups: 29
processing 2000 out of 28937 game_id 288887, cnt_null_matchups: 63
calling statsapi.player_stat_data for player_id: 592603
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 592603 stat
opposing_pitcher_id  is not of int64 type
calling statsapi.player_stat_data for player_id: 612538
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 612538 stat
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
processing 3000 out of 28937 game_id 318217, cnt_null_matchups: 158
opposing_pitcher_id  is not of int64 type
processing 4000 out of 28937 game_id 319211, cnt_null_matchups: 215
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
calling statsapi.player_stat_data for player_id

processing 8000 out of 28937 game_id 380921, cnt_null_matchups: 540
processing 9000 out of 28937 game_id 381918, cnt_null_matchups: 585
processing 10000 out of 28937 game_id 382914, cnt_null_matchups: 646


opposing_pitcher_id  is not of int64 type
calling statsapi.player_stat_data for player_id: 661470
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 661470 stat


opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
processing 11000 out of 28937 game_id 414508, cnt_null_matchups: 715


processing 12000 out of 28937 game_id 415512, cnt_null_matchups: 781
calling statsapi.player_stat_data for player_id: 661470
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 661470 stat
calling statsapi.player_stat_data for player_id: 629979
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 629979 stat
calling statsapi.player_stat_data for player_id: 666961
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 666961 stat
calling statsapi.player_stat_data for player_id: 667278
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 667278 stat
calling statsapi.player_stat_data for player_id: 662002
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 662002 stat
calling statsapi.player_stat_data for player_id: 666920
Exception: <class 'Exception'> while fetching player_stat_da

...

processing 16000 out of 28937 game_id 490889, cnt_null_matchups: 1103
processing 17000 out of 28937 game_id 491900, cnt_null_matchups: 1157
opposing_pitcher_id  is not of int64 type
calling statsapi.player_stat_data for player_id: 679619
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 679619 stat
calling statsapi.player_stat_data for player_id: 673934
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 673934 stat
calling statsapi.player_stat_data for player_id: 673919
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 673919 stat
calling statsapi.player_stat_data for player_id: 663314
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 663314 stat


processing 18000 out of 28937 game_id 529648, cnt_null_matchups: 1321
processing 19000 out of 28937 game_id 530650, cnt_null_matchups: 1366
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
processing 20000 out of 28937 game_id 531659, cnt_null_matchups: 1443
opposing_pitcher_id  is not of int64 type
calling statsapi.player_stat_data for player_id: 679548
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 679548 stat
calling statsapi.player_stat_data for player_id: 684830
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 684830 stat
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type


opposing_pitcher_id  is not of int64 type
processing 21000 out of 28937 game_id 564953, cnt_null_matchups: 1593
processing 22000 out of 28937 game_id 565377, cnt_null_matchups: 1654
opposing_pitcher_id  is not of int64 type
calling statsapi.player_stat_data for player_id: 679548
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 679548 stat
calling statsapi.player_stat_data for player_id: 685360
Exception: <class 'Exception'> while fetching player_stat_data
Error while getting away batter 685360 stat


processing 23000 out of 28937 game_id 631593, cnt_null_matchups: 1842
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
processing 24000 out of 28937 game_id 634589, cnt_null_matchups: 1998
opposing_pitcher_id  is not of int64 type
processing 25000 out of 28937 game_id 633747, cnt_null_matchups: 2954
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
processing 26000 out of 28937 game_id 632569, cnt_null_matchups: 3895
opposing_pitcher_id  is not of int64 type


processing 27000 out of 28937 game_id 663005, cnt_null_matchups: 4348
opposing_pitcher_id  is not of int64 type
processing 28000 out of 28937 game_id 662267, cnt_null_matchups: 4400
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
opposing_pitcher_id  is not of int64 type
cnt_null_matchups: 4442


In [252]:
game_matchup_dataframe = pd.concat(game_matchups).reset_index(drop=True) # [Hit_Real_Features].dropna()
print(f"Unique Games: {len(game_matchup_dataframe['game_id'].drop_duplicates())}")

Unique Games: 18910


In [254]:
game_matchup_dataframe['game_date'] = pd.to_datetime(game_matchup_dataframe['game_date'])
game_matchup_dataframe['game_datetime'] = pd.to_datetime(game_matchup_dataframe['game_datetime'])
game_matchup_dataframe['game_year'] = game_matchup_dataframe.game_date.dt.to_period('y')

In [255]:
game_matchup_dataframe.to_pickle('game_matchup_dataframe.pkl')